In [1]:
from train import *
from utils import *

In [2]:
#graphs: 
# distance matrix , error vs cluster,
# bid curve before cluster,bid curve after cluster,

In [3]:
train_file = "train_combined_pub_0313_0905.csv"
df = read_data(train_file)

In [4]:
columns_to_drop = ['MAXEOHSTATEOFCHARGE','PRODUCTBID_DESC','PRODUCTBID_MRID', 'MARKETPRODUCT_DESC', 'SCH_BID_Y2AXISDATA', 'MINEOHSTATEOFCHARGE', 'MAXEOHSTATEOFCHARGE', 'STARTTIME', 'STOPTIME', 'RESOURCE_TYPE', 'TIMEINTERVALSTART', 'TIMEINTERVALEND', 'MARKETPRODUCTTYPE', 'SCH_BID_TIMEINTERVALSTART_GMT', 'SCH_BID_TIMEINTERVALSTOP_GMT', 'SCH_BID_CURVETYPE']
df_droped = df.drop(columns=columns_to_drop, axis="columns")


In [5]:

HOUR = 0
RESOURCEBID_SEQ = 100651 #None #100651


In [6]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from draw_bid_curve import find_trace_bid_curves_for_multiple_days_2d

# create a subplot grid: 4 rows, 6 columns
rows = 4
cols = 6
fig_err = make_subplots(rows=rows, cols=cols, subplot_titles=[f"Hour {h}" for h in range(24)], horizontal_spacing=0.03, vertical_spacing=0.05)
fig_bid = make_subplots(rows=rows, cols=cols, subplot_titles=[f"Hour {h}" for h in range(24)], horizontal_spacing=0.03, vertical_spacing=0.05)


min_clusters=2
max_clusters= 10 #20

for HOUR in range(24):
    # print(f"Processing Hour {HOUR}...")
    df_filtered = filter_rows(df_droped, HOUR, RESOURCEBID_SEQ)
    df_train_feat = extract_feat_from_bid(df_filtered)
    df_train_feat_scaled = scale_feat(df_train_feat)

    row = (HOUR // cols) + 1
    col = (HOUR % cols) + 1   
    
    showlegend=True if HOUR == 0 else False
    kwargs = {'showlegend': showlegend}
    
    traces_err = find_trace_inter_intra_err(df_train_feat_scaled, min_clusters, max_clusters, **kwargs)
    for trace_err in traces_err:
        fig_err.add_trace(trace_err, row=row, col=col)
    
    result_dict = find_trace_bid_curves_for_multiple_days_2d(df_filtered, HOUR)
    traces_bid = result_dict['traces']
    for trace_bid in traces_bid:
        fig_bid.add_trace(trace_bid, row=row, col=col)


fig_height_factor = 250
fig_width_factor = fig_height_factor * 1.25        
fig_err.update_layout(
    height=rows*fig_height_factor,
    width=cols*fig_width_factor,
    title_text="Intercluster and intracluster errors by hour (X axis: Num of clusters, Y axis: Error(Sum of sqared distances))",
    legend=dict(
        orientation="h", #horizontal legend
        yanchor="top",
        y=1.05,
        xanchor="right",
        x=1
    ),
    hovermode="x unified", 
)

fig_bid.update_layout(
    height=rows*fig_height_factor,
    width=cols*fig_width_factor,
    title_text="Bidding curves by hour (X axis: Amount (MW), Y axis: Price ($))",
    showlegend=False,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        # y=1.05,
        xanchor="right",
        # x=1
    ),
    hovermode="x unified", 
)
# Show the figure
fig_err.show()
fig_bid.show()